In [9]:
import pandas as pd
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import ee
from functools import reduce
from sklearn.mixture import GaussianMixture
import numpy as np

pd.options.plotting.backend = "plotly"

In [10]:
geo_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Oeffentlich_zugaengliche_Strassenparkplaetze_OGD?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_pp_ogd'

with urlopen(geo_url) as response:
    geo_data = json.load(response)

df = pd.json_normalize(geo_data, "features")

df["lon"] = df["geometry.coordinates"].apply(lambda row: row[0])
df["lat"] = df["geometry.coordinates"].apply(lambda row: row[1])

In [11]:
# ee.Authenticate()
ee.Initialize()
dataset = ee.ImageCollection("SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL") 
# dataset = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") 
# dataset = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
# dataset = ee.ImageCollection("COPERNICUS/S1_GRD").select("angle")

In [12]:
def process_point(coords, scale=2, buffer=False):
    lon, lat = coords
    if buffer:
        poi = ee.Geometry.Point(lon, lat).buffer(buffer)
    else:
        poi = ee.Geometry.Point(lon, lat)

    ts = dataset.getRegion(poi, scale).getInfo()

    df = pd.DataFrame(ts[1:], columns=ts[0])
    df = df.dropna()
    df["datetime"] = pd.to_datetime(df["time"], unit="ms")

    return df

test = df["geometry.coordinates"][0:10].apply(lambda row: process_point(row))
test = reduce(lambda x, y: pd.concat([x, y]), test)

In [13]:
# sns.histplot(data = test2, x = "angle", bins=90)

# sns.pairplot(test2[[f"B{i}" for i in range(1, 9)] + ["B8A", "B9", "B11", "B12"]], kind="hist")

# grouped = test2.groupby(["longitude", "latitude"])

# fig = go.Figure()
# for name, group in grouped:
#     fig.add_scatter(name = str(name), x = group["B2"], y = group["B3"], mode="markers")

# fig.update_layout(
#     height=600,
#     width=900
# )

# fig.show()

In [18]:
num_test = test[["B", "G", "R", "N", "P"]]
# test3 = test2[[f"B{i}" for i in range(1, 4)]]
res = GaussianMixture(n_components=2, covariance_type="full").fit(num_test)
test["label"] = res.predict(num_test)
test

,id,longitude,latitude,time,B,G,R,N,P,datetime,label
1,s01_20160218T104052Z,8.515517,47.328774,1455792052000,3259.0,2310.0,1958.0,2949.0,5841.0,2016-02-18 10:40:52,1
2,s01_20160326T104737Z,8.515517,47.328774,1458989257000,1865.0,1145.0,852.0,994.0,2474.0,2016-03-26 10:47:37,0
3,s01_20160505T110038Z,8.515517,47.328774,1462446038000,1627.0,891.0,629.0,2294.0,2612.0,2016-05-05 11:00:38,0
4,s02_20151223T110337Z,8.515517,47.328774,1450868617000,1785.0,858.0,519.0,545.0,1978.0,2015-12-23 11:03:37,0
5,s02_20160411T111845Z,8.515517,47.328774,1460373525000,2063.0,1158.0,938.0,1953.0,3601.0,2016-04-11 11:18:45,0
...,...,...,...,...,...,...,...,...,...,...,...
5,s02_20151118T105647Z,8.546149,47.403909,1447844207000,1800.0,785.0,434.0,238.0,1787.0,2015-11-18 10:56:47,0
6,s02_20151213T110154Z,8.546149,47.403909,1450004514000,1884.0,866.0,524.0,313.0,1973.0,2015-12-13 11:01:54,0
7,s02_20151223T110337Z,8.546149,47.403909,1450868617000,1705.0,786.0,447.0,258.0,1765.0,2015-12-23 11:03:37,0
8,s02_20160317T111552Z,8.546149,47.403909,1458213352000,1883.0,915.0,572.0,483.0,2095.0,2016-03-17 11:15:52,0
